<a href="https://colab.research.google.com/github/SonOfADev/memes_vs_notes/blob/master/dnn_numpy_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
def initialization(next_layer_dims, prev_layer_dims):
    # random initialization because kaiming initialization was giving a problem 
    W  = np.random.randn(next_layer_dims, prev_layer_dims)*0.01
    b = np.zeros([next_layer_dims,1])
    return W, b

In [3]:
def forward_prop(A_prev, W, b):
    Z = np.dot(W, A_prev)+b
    
    return Z

In [4]:
def activation_function(Z, activation):
    if activation == 'sigmoid':
        A = 1/(1+np.exp(-Z))

    if activation == 'relu':
        A = np.maximum(Z, 0)

    return A 

In [5]:
def cost_function(AL, label):

    m = AL.shape[1]
    #epsilon = 0.00000001
    cost = -1/m*(np.sum(label*np.log(AL)+(1-label)*np.log(1-AL)))
    assert cost.shape == ()

    return cost

In [6]:
def relu_grad(dA, A):
    dZ = np.array(dA, copy=True)
   
    dZ[A == 0] = 0
    return dZ

In [7]:
def sigmoid_grad(A, Y):
    return A-Y

In [8]:
def backward_prop(dZ, A_prev, W ):

    m = A_prev.shape[1]
    dW = 1/m*(np.dot(dZ, A_prev.T))
    db = 1/m*np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db

In [24]:
def training_model(no_of_layers, nL, X, y, learning_rate, EPOCHS, val_x, val_y, batches):

    param= {}
    
    
    Z = {}
    #initializing parameters:
    for l in range(1, no_of_layers):
        param["W" + str(l)], param["b" + str(l)] = initialization(nL[l],nL[l-1])   
        W = param["W" + str(l)]
        x = W.shape

    for epochs in range(EPOCHS):
        correct = 0
        total_loss = 0
        batch_size = X.shape[1]//batches
        for b in range(batches):
            Y = y[b*batch_size:(b+1)*batch_size]
            results={"A0":X[:,b*batch_size:(b+1)*batch_size]}
            for l in range(1, no_of_layers-1):
                
                #initialization of parameters 
                W = param["W" + str(l)]
                b = param["b" + str(l)]

                #forward propogation 
                Z["Z"+ str(l)] = forward_prop(results["A" + str(l-1)], W, b)
                results["A"+ str(l)] = activation_function( Z["Z"+str(l)], activation="relu")
                ##################################

            Z["Z" + str(no_of_layers-1)] = forward_prop(results["A" + str(no_of_layers-2 )], param["W"+str(no_of_layers-1)], param["b"+str(no_of_layers-1)])

            results["A"+ str(no_of_layers-1)] = activation_function( Z["Z"+ str(no_of_layers-1)], activation="sigmoid") 
             ##############################
            
            #finding cost:

            cost = cost_function(results["A"+ str(no_of_layers-1)], Y)

        
            #backward prop :
            grads = {}

            AL = results["A"+ str(no_of_layers-1)] 
            #print(AL.shape)
            grads["dA"+str(no_of_layers-1)] = -(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
            dZ = grads["dA"+str(no_of_layers-1)]*sigmoid_grad(AL, Z["Z" + str(no_of_layers-1)])


            ###################
            for l in range(no_of_layers-1,0,-1):
                grads["dA"+str(l-1)],  grads["dW"+str(l)], grads["db"+str(l)] = backward_prop(dZ, results["A"+str(l-1)], param["W"+str(l)] )
                dZ = relu_grad(grads["dA"+str(l-1)],results["A"+str(l-1)])


            # updating the parameters:

            for l in range(1,no_of_layers):
                param["W"+str(l)] = param["W"+str(l)] - learning_rate*grads["dW"+str(l)]
                param["b"+str(l)] = param["b"+str(l)] - learning_rate*grads["db"+str(l)] 
            



            total_loss += cost_function(results["A"+ str(no_of_layers-1)], Y)

            AL = AL.reshape(-1)
            predictions =np.array([AL>0.5])
            correct += (predictions == Y).sum().item()

        print(f"EPOCH: {epochs+1}")
        print(f"accuracy_train:{0.1*correct}")
        print("")
        print(f"train_cost={total_loss/batches}")
        print("")
        validation(no_of_layers, nL, val_x, val_y, param)
        print("")

    return param


In [10]:
def validation(no_of_layers, nL, X, Y, param):

    results={"A0":X}   
    Z = {}
    for l in range(1, no_of_layers-1):
            #initialization of parameters 
            W = param["W" + str(l)]
            b = param["b" + str(l)]

            #forward propogation 
            Z["Z"+ str(l)] = forward_prop(results["A" + str(l-1)], W, b)
            results["A"+ str(l)] = activation_function( Z["Z"+str(l)], activation="relu")
            ##################################

    Z["Z" + str(no_of_layers-1)] = forward_prop(results["A" + str(no_of_layers-2 )], param["W"+str(no_of_layers-1)], param["b"+str(no_of_layers-1)])
    results["A"+ str(no_of_layers-1)] = activation_function( Z["Z"+ str(no_of_layers-1)], activation="sigmoid")
 
 ##############################
    
    #finding cost:
    AL = results["A"+ str(no_of_layers-1)]

    cost = cost_function(results["A"+ str(no_of_layers-1)], Y)

    print(f"cost_valid ={cost}")

    

    predictions = [AL>0.5]
    total = Y.shape[0]
    correct = (predictions == Y).sum().item()
    print(f"acc_valid:{100*correct/total}")



In [11]:
import PIL 
from PIL import Image
import os

 
folder = "/content/drive/My Drive/Memes_vs_Notes/"
classes = ["Memes", "Notes"]

In [12]:
from sklearn.utils import shuffle
import cv2
def load_data(folder, classes, IMG_HEIGHT, IMG_WIDTH):
    data = []
    labels = []
    label = -1
    for y in classes:
        label += 1
        path = folder + str(y)+"/"
        for x in os.listdir(path):
            img = cv2.imread((path + str(x)),0)
            img = cv2.resize(img,(IMG_WIDTH, IMG_HEIGHT))
            data.append(img)      
            labels.append(label)

    data, labels = shuffle(data, labels,random_state=0)
    # return train_data, train_labels, valid_data, valid_labels, test_data, test_labels
    return data, labels

In [14]:
IMG_HEIGHT = 50
IMG_WIDTH = 50

# train_data, train_labels, valid_data, valid_labels, test_data, test_labels = load_data(folder, classes, IMG_HEIGHT, IMG_WIDTH)
data, labels = load_data(folder, classes, IMG_HEIGHT, IMG_WIDTH)

In [15]:

train_data = data[:1000]
train_labels = labels[:1000]
valid_data = data[1000:1300]
valid_labels = labels[1000:1300]
test_data = data[1300:1600]
test_labels = labels[1300:1600]
np.shape(train_data)

train_labels = np.array(train_labels)

valid_labels = np.array(valid_labels)

test_labels = np.array(test_labels)


data = np.array(data, dtype="float")
labels = np.array(labels)


In [16]:
train_data = np.array(train_data)

valid_data = np.array(valid_data)

test_data = np.array(test_data)

train_data = (train_data.reshape(train_data.shape[0],-1)).T

valid_data = valid_data.reshape(valid_data.shape[0], -1).T

test_data = test_data.reshape(test_data.shape[0],-1).T

In [17]:

total_train = np.sum([x==1 for x in train_labels])
total_valid = np.sum([x==1 for x in valid_labels])
total_test = np.sum([x==1 for x in test_labels])
total_train, total_valid, total_test  # to check for bias in train data this was even done for valid_data and test_datra

(494, 158, 148)

In [18]:
train_data = train_data/255.0
valid_data = valid_data/255.0

In [19]:

type(train_data[0,0]), train_labels.shape, train_data.shape, valid_data.shape

(numpy.float64, (1000,), (2500, 1000), (2500, 300))

In [25]:
nL = [train_data.shape[0],320, 128,1 ] 
params = training_model(4, nL, train_data, train_labels, 0.001, 25, valid_data, valid_labels, 50)

EPOCH: 1
accuracy_train:50.300000000000004

train_cost=0.6930225580581246

cost_valid =0.6930296138786021
acc_valid:47.333333333333336

EPOCH: 2
accuracy_train:54.6

train_cost=0.6929016433720276

cost_valid =0.6929149052502787
acc_valid:51.333333333333336

EPOCH: 3
accuracy_train:61.6

train_cost=0.6927875638153583

cost_valid =0.6928042832177438
acc_valid:58.666666666666664

EPOCH: 4
accuracy_train:70.60000000000001

train_cost=0.6926769075398868

cost_valid =0.6926947216483705
acc_valid:67.33333333333333

EPOCH: 5
accuracy_train:74.7

train_cost=0.6925635092176949

cost_valid =0.6925842808967102
acc_valid:72.0

EPOCH: 6
accuracy_train:75.8

train_cost=0.6924488388484413

cost_valid =0.692478588764091
acc_valid:74.0

EPOCH: 7
accuracy_train:77.30000000000001

train_cost=0.6923362832829731

cost_valid =0.6923746478937592
acc_valid:75.66666666666667

EPOCH: 8
accuracy_train:77.9

train_cost=0.6922230139683283

cost_valid =0.6922682889959533
acc_valid:77.0

EPOCH: 9
accuracy_train:78.4
